In [ ]:
import pandas as pd
from tensorflow.keras.utils import to_categorical

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import numpy as np

In [ ]:
train_data= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data= pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_labels= pd.DataFrame()
train_labels= train_data['label'].values

In [ ]:
train_data1=pd.DataFrame()
train_data1= train_data.iloc[:, 1:]

In [ ]:
train_data1= train_data1.astype('float32')/255
test_data= test_data.astype('float32')/255

In [ ]:
train_labels= to_categorical(train_labels)

In [ ]:
def create_cnn_model():
    model_cnn= Sequential()
    model_cnn.add(Conv2D(32, kernel_size= (3,3), activation= 'relu', input_shape= (28,28,1)))
    model_cnn.add(MaxPooling2D(pool_size= (2,2)))
    model_cnn.add(Conv2D(64, kernel_size= (3,3), activation= 'relu'))
    model_cnn.add(MaxPooling2D(pool_size= (2,2)))
    model_cnn.add(Flatten())
    model_cnn.add(Dense(384, activation= 'relu'))
    model_cnn.add(Dense(384, activation= 'relu'))
    model_cnn.add(Dense(10, activation = 'softmax'))
    model_cnn.compile(optimizer= 'adam', loss= 'categorical_crossentropy', metrics= ['accuracy'])
    return model_cnn

In [ ]:
train_labels = train_data['label'].values
train_data = train_data.drop(columns=['label'])
train_data_reshaped = train_data.values.reshape(-1, 28, 28, 1)
train_data_reshaped = train_data_reshaped.astype('float32') / 255.0
train_labels = to_categorical(train_labels)

In [ ]:
if train_data_reshaped.shape[0] == 0:
    print("Training data is empty. Please check data loading and preprocessing.")
else:
    model_cnn = create_cnn_model()
    history = model_cnn.fit(train_data_reshaped, train_labels, epochs=10, batch_size=128, validation_split=0.25)
    validation_accuracy = history.history['val_accuracy'][-1]
    print(f"Validation Accuracy: {validation_accuracy * 100}")

In [ ]:
test_data_arr= test_data.to_numpy()
test_data_reshaped= test_data_arr.reshape((-1,28,28,1))
predictions= model_cnn.predict(test_data_reshaped)

In [ ]:
predicted_classes= np.argmax(predictions, axis=1)

In [ ]:
results= pd.Series(predicted_classes, name="Label")
submission= pd.concat([pd.Series(range(1,28001), name="ImageId"), results], axis=1)
submission.to_csv("submission.csv", index=False)